# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Create worspace and experiment

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'ml-capstone-automl'

experiment=Experiment(ws, experiment_name)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
dataset = Dataset.get_by_name(ws, name = 'Diabetes-predict')

In [4]:
dataset.take(5).to_pandas_dataframe()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


## Connect to compute target

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook240238"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diabetes",  
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-capstone-automl,AutoML_cb27844e-f412-4385-965f-dfd40bcd7d6f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_cb27844e-f412-4385-965f-dfd40bcd7d6f',
 'target': 'notebook240238',
 'status': 'Completed',
 'startTimeUtc': '2023-08-14T10:59:47.821667Z',
 'endTimeUtc': '2023-08-14T11:10:41.865615Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook240238',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-capstone-automl","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-240238","workspace_name":"quick-starts-ws-240238","region":"westeurope","compute_target":"notebook

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Retrieve best model from Pipeline Run
best_automl_run, fitted_model = remote_run.get_output()
print(best_automl_run.get_details())
print(best_automl_run.get_metrics())

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

{'runId': 'AutoML_cb27844e-f412-4385-965f-dfd40bcd7d6f_21', 'target': 'notebook240238', 'status': 'Completed', 'startTimeUtc': '2023-08-14T11:04:42.320016Z', 'endTimeUtc': '2023-08-14T11:05:18.075776Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': 'e478544eb8734c07a1943f5bdf4e6605be6943df', 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 0.5, "eta": 0.2, "gamma": 0, "max_depth": 7, "max_leaves": 7, "n_estimators": 25, "objective": "reg:logistic", "reg_alpha": 0, "reg_lambda": 0.20833333333333334, "subsample": 1, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "e478544eb8734c07a19

In [17]:
best_automl_run.register_model(model_name='AutoML_best_run', model_path='.')

Model(workspace=Workspace.create(name='quick-starts-ws-240238', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-240238'), name=AutoML_best_run, id=AutoML_best_run:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
from azureml.core import Model 
import joblib

joblib.dump(fitted_model, 'best-automl-model.pkl')

model = Model.register(
    workspace=ws,
    model_name='best-automl-model',
    model_path='./best-automl-model.pkl'
)

Registering model best-automl-model


In [38]:
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Environment, ScriptRunConfig

#create environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#define inference config 
inference_config = InferenceConfig(
    entry_script='score.py',
    source_directory=".",
    environment=sklearn_env
)

#define deployment config
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    auth_enabled=True,
    enable_app_insights=True
)

service = Model.deploy(
    ws,
    name = 'automl-service',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite = True
)
service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-14 15:31:20+00:00 Creating Container Registry if not exists.
2023-08-14 15:31:20+00:00 Registering the environment.
2023-08-14 15:31:21+00:00 Use the existing image.
2023-08-14 15:31:22+00:00 Generating deployment configuration.
2023-08-14 15:31:22+00:00 Submitting deployment to compute.
2023-08-14 15:31:25+00:00 Checking the status of deployment automl-service..
2023-08-14 15:34:18+00:00 Checking the status of inference endpoint automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [39]:
service.get_logs

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-240238', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-240238'), name=automl-service, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://995cd954-7543-4c92-93ca-b434a10959c7.westeurope.azurecontainer.io/score, tags=None, properties={'hasInferenceSchema': 'True', 'hasHttps': 'False', 'authEnabled': 'True'}, created_by={'userObjectId': '8c14c3f9-1132-4705-9c75-ed5fed51e496', 'userPuId': '10032002D5BE822A', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/660b3398-b80e-49d2-bc5b-ac1dc93b5254/', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 240238', 'upn': 'odl_user_240238@udacitylabs.onmicrosoft.com'})>

In [ ]:
published_model = remote_run.publish_pipeline(
    name="Diabetes Train", description="Training diabetes pipeline", version="1.0")

published_model

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [60]:
data["data"]

[{'gender': 'Female',
  'age': 55,
  'hypertension': 0,
  'heart_disease': 1,
  'smoking_habits': 'never',
  'bmi': 24.0,
  'HbA1c_level': 5.1,
  'blood_glucose_level': 140}]

In [61]:
import requests
import json

scoring_uri = service.scoring_uri

key, _ = service.get_keys()

data = {
        "data": [{
              "gender": "Female",
              "age": 55,
              "hypertension": 0,
              "heart_disease": 1,
              "smoking_habits": "never",
              "bmi": 24.0,
              "HbA1c_level": 5.1,
              "blood_glucose_level": 140
            }]
        
    }

input_data = json.dumps(data["data"])
headers = {"Content-Type": "application/json"}
headers['Authorization']=f"Bearer {key}"

response = requests.post(scoring_uri, data=input_data, headers=headers)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
rest_endpoint.delete()

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
